In [1]:
import pandas as pd
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from requests import get
from tika import parser
import urllib.request

In [2]:
def download_save_img(url, file_name):
    urllib.request.urlretrieve(url, file_name)

def download_pdf(url, file_name):
    with open(file_name, "wb") as file:
        response = get(url)
        file.write(response.content)
        
def read_and_write(file_name, text_name):
    with open(text_name, "w", encoding='utf8') as writer:
        parsed = parser.from_file(file_name)
        text = parsed['content']
        lines = text.split('\n')
        full_text = []
        for line in lines:
            if line == '':
                continue
            else:
                writer.write(line.replace('�', ' ')+'\n')
                
def read_and_return(file_name):
    parsed = parser.from_file(file_name)
    text = parsed['content']
    lines = text.split('\n')
    full_text = ""
    for line in lines:
        if line == '':
            continue
        else:
            full_text+=(line.replace('�', ' ')+'\n')
    return full_text

In [3]:
data_detail = pd.read_excel('csv/pill_details.xls')
data_detail = data_detail[['품목일련번호', '허가일자', '원료성분', '마약류분류', '완제원료구분', '포장단위', '보험코드', '전문일반', '총량', '취소상태',
       '변경내용', '등록자id', '첨부문서', '제심사기간', '효능효과', '저장방법', '신약여부', '유효기간',
       '용법용량', '첨가제명', '성상', '재심사대상', '허가및신고구분',
       '취소일자', '표준코드', '주의사항', '주성분명']]

data_list = pd.read_csv('csv/pill_list.csv')
data_list = data_list[['품목일련번호', '품목명', '업소일련번호', '업소명', '큰제품이미지', '표시앞', '표시뒤', '의약품제형',
       '색상앞', '색상뒤', '분할선앞', '분할선뒤', '크기장축', '크기단축', '크기두께', '이미지생성일자(약학정보원)',
       '분류번호', '분류명', '전문일반구분', '품목허가일자', '제형코드명', '표기내용앞', '표기내용뒤', '표기이미지앞',
       '표기이미지뒤', '표기코드앞', '표기코드뒤', '변경일자']]

In [4]:
new_Data = pd.merge(data_list, data_detail, on='품목일련번호')

In [5]:
img = '큰제품이미지'
eff = '효능효과'
use = '용법용량'
war = '주의사항'

In [6]:
row_num = len(new_Data.index)
print("total row number:",row_num)

for idx in range(5):
    pill_id = new_Data.loc[idx]['품목일련번호']
    print(idx,"Pill ID: "+str(pill_id)+"-"*20)
    pill_img = new_Data.loc[idx][img]
    pill_eff = new_Data.loc[idx][eff]
    pill_use = new_Data.loc[idx][use]
    pill_war = new_Data.loc[idx][war]
    
    ## 큰제품이미지
    file_name = str(pill_id)+'_img'
    download_save_img(pill_img, 'img/'+file_name+'.jpg')
    new_Data.iloc[idx,4] = file_name+'.jpg'
    print("image Done.")
    
    ## 효능효과
    file_name = str(pill_id)+'_effect'
    download_pdf(pill_eff, 'pdf/'+file_name+'.pdf')
    text = read_and_return('pdf/'+file_name+'.pdf')
    new_Data.iloc[idx,41] = text
    print(new_Data.loc[idx][eff])
    print("effect Done.")
    
    ## 용법용량
    file_name = str(pill_id)+'_usage'
    download_pdf(pill_use, 'pdf/'+file_name+'.pdf')
    text = read_and_return('pdf/'+file_name+'.pdf')
    new_Data.iloc[idx,45] = text
    print(new_Data.loc[idx][use])
    print("usage Done.")
    
    ## 주의사항
    file_name = str(pill_id)+'_warning'
    download_pdf(pill_war, 'pdf/'+file_name+'.pdf')
    text = read_and_return('pdf/'+file_name+'.pdf')
    new_Data.iloc[idx,52] = text
    print(new_Data.loc[idx][war])
    print("warning Done.")

total row number: 21657
0 Pill ID: 199304470--------------------
image Done.
효능효과
1. 주효능 효과
1) 말초동맥 순환장애(간헐성 파행증)의 치료
2) 어지러움, 혈관성 및 퇴행성 이명
3) 이명, 두통, 기억력감퇴, 집중력장애, 우울감, 어지러움 등의 치매성 증상을 수반하는 기질성 뇌기능장애
의 치료

effect Done.
용법용량
성인
1) 말초동맥순환장애, 어지러움, 이명(귀울림): 은행엽건조엑스로서 1회 40밀리그람 1일 3회 또는 1회 80밀
리그람, 1일 2회 경구투여한다(먹는다).
2) 기질성 뇌기능장애 : 은행엽건조엑스로서 1회 40~80밀리그람 1일 3회 또는 1회 120밀리그람 1일 2회 
경구투여한다(먹는다).
연령(나이), 증상에 따라 적절히 증감한다

usage Done.
사용상의주의사항
1. 다음과 같은 사람은 이 약을 복용(사용)하지 말 것
1) 이 약에 과민증 환자
2) 12세 이하의 소아(소아의 사용경험이 충분하지 않으므로 12세 이하의 소아에는 투여하지 않는다.)
3) 임부 (동물시험에서 기형발생은 없었으나 임신중의 투여에 대한 충분한 사용경험이 적다.)
4) 이 약은 유당을 함유하고 있으므로, 갈락토오스 불내성(galactose intolerance), Lapp 유당분해효소 결핍
증(Lapp lactase deficiency) 또는 포도당-갈락토오스 흡수장애(glucose-galactose malabsorption) 등
의 유전적인 문제가 있는 환자
2. 다음과 같은 사람은 이 약을 복용(사용)하기 전에 의사, 치과의사, 약사와 상의할 것
1) 출혈 장애가 있거나 출혈의 위험을 증가시키는 약물(항응고제(예: 펜프로쿠몬, 와파린)), 항혈소판제(예:클
로피도그렐, 아세틸살리실산 및 기타 비스테로이드성 항염증제))을 복용하고 있는 환자
2) 수유부(모유로 이행여부가 알려지지 않음)
3. 다음과 같은 경우 이 약의 복용(사용)을 즉각 중단하고 의사, 치과의사, 약사와

image Done.
효능효과
1. 계절성 및 다년성 알레르기성 비염[코염], 알레르기성 결막염, 만성 특발성[원인 불명의] 두드러기, 피부소양
증[가려움증]
2. 하이드로코티손 외용제와 병용[함께 복용]에 의한 습진, 피부염

effect Done.
용법용량
1. 성인 및 6세 이상의 소아 : 세티리진염산염으로서 1일 1회 10mg 취침 전에 경구 투여하십시오. [복용하십시
오.]
2. 이상반응에 민감한 환자의 경우는 5mg씩을 아침, 저녁에 분할 투여하십시오.
3. 연령[나이], 증상에 따라 적절히 증감하십시오.
4. 중등도 ∼ 중증[심한증상]의 신장애[신장장애] 환자 : 투여 간격은 환자의 신기능에 따라 조절되어야 합니
다. 다음 표를 참조하여 용량을 조절합니다. 이 용량 표를 사용하기 위해서는 mL/min 단위의 크레아티닌 청소
율 (CLcr)이 필요하다. CLcr (mL/min)은 다음 공식을 이용하여 혈청 크레아티닌 (mg/dL) 측정치로부터 계산
된다 :
CLcr =
[140 ­ 연령[나이] (세)] × 체중 (kg)
(여성인 경우 × 0.85)
72 × 혈청 크레아티닌 (mg/dL)
신기능이 손상된 성인 환자에 대한 용량 조절
구분 크레아티닌 청소율 (mL/min) 용량 및 빈도
정상 ≥80 1일 1회, 10 mg
경증 [가벼운 증상] 50 ­ 79 1일 1회, 10 mg
중등도 30 ­ 49 1일 1회, 5 mg
중증 [심한 증상] < 30 매 2일마다 1회, 5 mg
말기 단계의 신 질환을 가진 투석중인 환자 <10 금기
신장애[신장장애]를 가진 소아 환자는 환자의 신 청소율과 체중을 고려하여 개별적으로 용량을 조절해야 한
다.

usage Done.
사용상의주의사항
1. 다음과 같은 사람은 이 약을 복용(사용)하지 말 것
1) 이 약 및 히드록시진 또는 피페라진 유도체에 과민증 및 그 병력이 있는 환자
2) 신부전 환자 (크레아티닌 청소율 < 10mL/min)
3) 임부 및 임신하고 있을 가능성이 있는 부인, 수유부
4) 6세 미만의 유아


In [8]:
new_Data.to_excel("csv/pill_information.xlsx")

new_Data.to_csv("csv/pill_information.csv", "w")

In [2]:
def download_save_img(url, file_name):
    urllib.request.urlretrieve(url, file_name)

In [4]:
download_save_img("http://jchoi315.cafe24.com/wp-content/uploads/2019/03/이화로고_00111-1-300x76.jpg", "EWHA_LOGO1.png")

UnicodeEncodeError: 'ascii' codec can't encode characters in position 32-35: ordinal not in range(128)